# 会话管理调试笔记本

本笔记本用于调试和测试会话管理功能：
- Session 创建与管理
- 消息历史管理
- 会话过期与清理
- Prompt 模板系统
- 上下文窗口管理

In [1]:
# 添加项目根目录到路径
import sys
import os
sys.path.insert(0, os.path.dirname(os.getcwd()))

# 设置必要的环境变量
os.environ.setdefault("TELEGRAM_BOT_TOKEN", "test_token")
os.environ.setdefault("OPENAI_API_KEY", "test_key")

'test_key'

## 1. 导入会话管理模块

In [2]:
from src.conversation import (
    SessionManager,
    Session,
    Message,
    get_session_manager,
    PromptTemplate,
    PromptTemplateManager,
    get_template_manager,
    ContextManager,
    ContextWindow,
    get_context_manager
)

print("✅ 会话管理模块导入成功")

✅ 会话管理模块导入成功


## 2. 会话管理器 (SessionManager) 测试

In [3]:
# 创建会话管理器
session_manager = SessionManager(
    default_expiry_minutes=30,     # 会话30分钟过期
    max_sessions_per_user=5,       # 每用户最多5个会话
    max_messages_per_session=100   # 每会话最多100条消息
)

print(f"会话管理器创建成功")
print(f"  默认过期时间: {session_manager.default_expiry_minutes} 分钟")
print(f"  每用户最大会话数: {session_manager.max_sessions_per_user}")
print(f"  每会话最大消息数: {session_manager.max_messages_per_session}")

2026-01-22 11:30:27.817 | INFO     | src.conversation.session_manager:__init__:175 - SessionManager initialized: expiry=30min, max_sessions=5, max_messages=100


会话管理器创建成功
  默认过期时间: 30 分钟
  每用户最大会话数: 5
  每会话最大消息数: 100


In [4]:
# 创建新会话
session = session_manager.create_session(
    user_id="user_001",
    bot_id="soulmate_bot",
    system_prompt="你是一个温暖的情感陪伴助手，请用关怀的语气回复用户。"
)

print(f"会话创建成功:")
print(f"  会话 ID: {session.session_id}")
print(f"  用户 ID: {session.user_id}")
print(f"  Bot ID: {session.bot_id}")
print(f"  消息数: {len(session.messages)}")
print(f"  是否活跃: {session.is_active}")

2026-01-22 11:30:39.156 | INFO     | src.conversation.session_manager:create_session:236 - Created session: user_001:soulmate_bot:485f9989


会话创建成功:
  会话 ID: user_001:soulmate_bot:485f9989
  用户 ID: user_001
  Bot ID: soulmate_bot
  消息数: 1
  是否活跃: True


In [5]:
# 添加消息到会话
session.add_user_message("你好，我今天心情不太好")
session.add_assistant_message("我理解你的感受。能告诉我发生了什么吗？我在这里倾听。")
session.add_user_message("工作压力太大了")
session.add_assistant_message("工作压力确实让人很疲惫。你有没有试过一些放松的方法呢？")

print(f"当前消息数: {len(session.messages)}")
print("\n消息列表:")
for i, msg in enumerate(session.messages):
    role_emoji = "🤖" if msg.role == "assistant" else "👤" if msg.role == "user" else "⚙️"
    print(f"  {role_emoji} [{msg.role}]: {msg.content[:50]}..." if len(msg.content) > 50 else f"  {role_emoji} [{msg.role}]: {msg.content}")

当前消息数: 5

消息列表:
  ⚙️ [system]: 你是一个温暖的情感陪伴助手，请用关怀的语气回复用户。
  👤 [user]: 你好，我今天心情不太好
  🤖 [assistant]: 我理解你的感受。能告诉我发生了什么吗？我在这里倾听。
  👤 [user]: 工作压力太大了
  🤖 [assistant]: 工作压力确实让人很疲惫。你有没有试过一些放松的方法呢？


In [6]:
# 获取 LLM 格式的消息
llm_messages = session.get_messages_for_llm()

print("LLM 格式消息:")
for msg in llm_messages:
    print(f"  {msg}")

LLM 格式消息:
  {'role': 'system', 'content': '你是一个温暖的情感陪伴助手，请用关怀的语气回复用户。'}
  {'role': 'user', 'content': '你好，我今天心情不太好'}
  {'role': 'assistant', 'content': '我理解你的感受。能告诉我发生了什么吗？我在这里倾听。'}
  {'role': 'user', 'content': '工作压力太大了'}
  {'role': 'assistant', 'content': '工作压力确实让人很疲惫。你有没有试过一些放松的方法呢？'}


In [7]:
# 设置和获取上下文数据
session.set_context("mood", "stressed")
session.set_context("topic", "work_pressure")
session.set_context("interaction_count", 2)

print("会话上下文:")
print(f"  心情: {session.get_context('mood')}")
print(f"  话题: {session.get_context('topic')}")
print(f"  交互次数: {session.get_context('interaction_count')}")
print(f"  不存在的键: {session.get_context('not_exists', '默认值')}")

会话上下文:
  心情: stressed
  话题: work_pressure
  交互次数: 2
  不存在的键: 默认值


In [8]:
# 测试 get_or_create_session
# 第一次调用 - 创建新会话
session2 = session_manager.get_or_create_session("user_002", "assistant_bot")
print(f"创建新会话: {session2.session_id}")

# 第二次调用 - 应该返回同一个会话
session2_again = session_manager.get_or_create_session("user_002", "assistant_bot")
print(f"获取已有会话: {session2_again.session_id}")
print(f"是同一个会话: {session2.session_id == session2_again.session_id}")

2026-01-22 11:31:08.588 | INFO     | src.conversation.session_manager:create_session:236 - Created session: user_002:assistant_bot:f0e68b9b


创建新会话: user_002:assistant_bot:f0e68b9b
获取已有会话: user_002:assistant_bot:f0e68b9b
是同一个会话: True


In [9]:
# 获取统计信息
stats = session_manager.get_stats()

print("会话管理器统计:")
for key, value in stats.items():
    print(f"  {key}: {value}")

会话管理器统计:
  total_sessions: 2
  active_sessions: 2
  total_users: 2
  total_messages: 5


## 3. Prompt 模板管理器测试

In [10]:
# 获取模板管理器
template_manager = get_template_manager()

# 列出所有模板
templates = template_manager.list_templates()

print(f"已加载 {len(templates)} 个模板:")
for template in templates:
    print(f"  - {template.name} ({template.category}, {template.language})")
    print(f"    描述: {template.description[:50]}..." if len(template.description) > 50 else f"    描述: {template.description}")

2026-01-22 11:31:13.424 | INFO     | src.conversation.prompt_template:__init__:230 - PromptTemplateManager initialized with 5 templates


已加载 5 个模板:
  - emotional_companion (companion, zh)
    描述: 情感陪伴机器人的系统提示词
  - tech_assistant (assistant, zh)
    描述: 技术助手机器人的系统提示词
  - customer_service (service, zh)
    描述: 客服机器人的系统提示词
  - general_assistant (general, zh)
    描述: 通用助手的系统提示词
  - emotional_companion_en (companion, en)
    描述: Emotional companion bot system prompt (English)


In [11]:
# 获取并渲染情感陪伴模板
emotional_template = template_manager.get_template("emotional_companion")

if emotional_template:
    print(f"模板名称: {emotional_template.name}")
    print(f"模板变量: {emotional_template.variables}")
    
    # 渲染模板
    rendered = emotional_template.render(
        bot_name="暖心助手",
        user_name="小明",
        interaction_count=5
    )
    
    print(f"\n渲染后的内容:")
    print(rendered)

模板名称: emotional_companion
模板变量: ['user_name', 'bot_name', 'interaction_count']

渲染后的内容:
你是一个温柔、善解人意的情感陪伴助手。

角色设定：
- 名字：暖心助手
- 性格：温暖、耐心、富有同理心
- 目标：倾听用户的心声，提供情感支持和陪伴

沟通原则：
1. 始终使用温暖、关怀的语气
2. 认真倾听并理解用户的情绪
3. 不评判，只给予支持和鼓励
4. 适时提供实用的情绪调节建议
5. 保持适当的边界感

用户信息：
- 用户名：小明
- 历史互动次数：5

请用温暖、关怀的语气回复用户，让他们感受到被理解和支持。


In [12]:
# 创建自定义模板
custom_template = PromptTemplate(
    name="customer_service_cn",
    content="""你是{{company_name}}的智能客服助手 {{bot_name}}。

服务准则：
1. 始终保持礼貌和专业
2. 快速理解用户需求
3. 提供准确的信息

当前用户：{{user_name}}
请用专业的态度为用户提供帮助。""",
    description="中文客服助手模板",
    category="service",
    language="zh"
)

# 注册模板
template_manager.register_template(custom_template)

# 使用自定义模板
result = template_manager.render_template(
    "customer_service_cn",
    company_name="科技有限公司",
    bot_name="小智",
    user_name="张先生"
)

print("渲染结果:")
print(result)

2026-01-22 11:31:28.991 | INFO     | src.conversation.prompt_template:register_template:245 - Registered template: customer_service_cn


渲染结果:
你是科技有限公司的智能客服助手 小智。

服务准则：
1. 始终保持礼貌和专业
2. 快速理解用户需求
3. 提供准确的信息

当前用户：张先生
请用专业的态度为用户提供帮助。


In [13]:
# 使用便捷方法创建系统提示词
system_prompt = template_manager.create_system_prompt(
    template_name="general_assistant",
    bot_name="智能助手",
    user_name="用户"
)

print("生成的系统提示词:")
print(system_prompt)

生成的系统提示词:
你是一个智能助手，名叫智能助手。

你的任务是帮助用户完成各种任务，回答问题，提供信息和建议。

请根据用户的需求，提供专业、有帮助的回复。


## 4. 上下文窗口管理器测试

In [14]:
# 创建上下文管理器
context_manager = ContextManager(
    default_max_tokens=4096,
    default_reserved_tokens=1000
)

print("上下文管理器创建成功")
print(f"  默认最大 Token: {context_manager.default_max_tokens}")
print(f"  默认保留 Token: {context_manager.default_reserved_tokens}")

上下文管理器创建成功
  默认最大 Token: 4096
  默认保留 Token: 1000


In [15]:
# 测试模型限制获取
models = ["gpt-4", "gpt-4-turbo", "gpt-4o", "claude-3-opus", "unknown-model"]

print("模型上下文限制:")
for model in models:
    limit = context_manager.get_model_limit(model)
    print(f"  {model}: {limit:,} tokens")

模型上下文限制:
  gpt-4: 8,192 tokens
  gpt-4-turbo: 128,000 tokens
  gpt-4o: 128,000 tokens
  claude-3-opus: 200,000 tokens
  unknown-model: 4,096 tokens


In [16]:
# 创建上下文窗口
context = context_manager.create_context(
    user_id="user_001",
    bot_id="soulmate_bot",
    model="gpt-4",
    system_prompt="你是一个温暖的助手。"
)

print(f"上下文窗口创建成功")
print(f"  最大 Token: {context.max_tokens}")
print(f"  可用 Token: {context.available_tokens}")
print(f"  当前 Token 估算: {context.get_total_tokens()}")

2026-01-22 11:32:03.432 | DEBUG    | src.conversation.context_manager:create_context:218 - Created context: user_001:soulmate_bot (max_tokens=8192)


上下文窗口创建成功
  最大 Token: 8192
  可用 Token: 7192
  当前 Token 估算: 6


In [17]:
# 添加消息并测试自动截断
context.add_message("user", "你好，请帮我写一篇关于人工智能的文章。")
context.add_message("assistant", "好的，我很乐意帮您写一篇关于人工智能的文章。请问您希望文章重点介绍哪些方面？")
context.add_message("user", "主要介绍机器学习和深度学习的发展历史。")

print(f"当前消息数: {len(context.messages)}")
print(f"当前 Token 估算: {context.get_total_tokens()}")

# 获取 LLM 格式的消息
llm_messages = context.get_messages_for_llm()
print(f"\nLLM 消息数（含 system）: {len(llm_messages)}")
for msg in llm_messages:
    print(f"  [{msg['role']}]: {msg['content'][:40]}...")

当前消息数: 3
当前 Token 估算: 65

LLM 消息数（含 system）: 4
  [system]: 你是一个温暖的助手。...
  [user]: 你好，请帮我写一篇关于人工智能的文章。...
  [assistant]: 好的，我很乐意帮您写一篇关于人工智能的文章。请问您希望文章重点介绍哪些方面？...
  [user]: 主要介绍机器学习和深度学习的发展历史。...


In [18]:
# 获取上下文管理器统计
stats = context_manager.get_stats()

print("上下文管理器统计:")
for key, value in stats.items():
    print(f"  {key}: {value}")

上下文管理器统计:
  total_contexts: 1
  total_messages: 3
  total_tokens_estimated: 65


## 5. 会话数据导出

In [19]:
# 将会话导出为字典
import json

session_data = session.to_dict()

print("会话数据导出:")
print(json.dumps(session_data, indent=2, ensure_ascii=False, default=str))

会话数据导出:
{
  "session_id": "user_001:soulmate_bot:485f9989",
  "user_id": "user_001",
  "bot_id": "soulmate_bot",
  "messages": [
    {
      "role": "system",
      "content": "你是一个温暖的情感陪伴助手，请用关怀的语气回复用户。",
      "timestamp": "2026-01-22T03:30:39.156558+00:00",
      "metadata": {}
    },
    {
      "role": "user",
      "content": "你好，我今天心情不太好",
      "timestamp": "2026-01-22T03:30:48.417119+00:00",
      "metadata": {}
    },
    {
      "role": "assistant",
      "content": "我理解你的感受。能告诉我发生了什么吗？我在这里倾听。",
      "timestamp": "2026-01-22T03:30:48.417157+00:00",
      "metadata": {}
    },
    {
      "role": "user",
      "content": "工作压力太大了",
      "timestamp": "2026-01-22T03:30:48.417185+00:00",
      "metadata": {}
    },
    {
      "role": "assistant",
      "content": "工作压力确实让人很疲惫。你有没有试过一些放松的方法呢？",
      "timestamp": "2026-01-22T03:30:48.417209+00:00",
      "metadata": {}
    }
  ],
  "context": {
    "mood": "stressed",
    "topic": "work_pressure",
    "interaction_count": 2
  